## Find the best neighborhood in Madrid to open a...

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [83]:
df = pd.read_csv('barrios_municipio_madrid.csv',sep= ';', encoding='Latin 1')
df.head()

,barrio_codigo,barrio_nombre,distrito_codigo,distrito_nombre,municipio_codigo,municipio_nombre,superficie_km2,densidad_por_km2
0,796011,Palacio,79601,Centro,796,Madrid,1.46,15323.29
1,796012,Embajadores,79601,Centro,796,Madrid,1.03,43345.63
2,796013,Cortes,79601,Centro,796,Madrid,0.59,17850.85
3,796014,Justicia,79601,Centro,796,Madrid,0.75,21866.67
4,796015,Universidad,79601,Centro,796,Madrid,0.93,33051.61


In [84]:
df.sort_values('distrito_nombre').head(20)

,barrio_codigo,barrio_nombre,distrito_codigo,distrito_nombre,municipio_codigo,municipio_nombre,superficie_km2,densidad_por_km2
9,796024,Legazpi,79602,Arganzuela,796,Madrid,1.37,13826.28
12,796027,Atocha,79602,Arganzuela,796,Madrid,0.79,1401.27
11,796026,Palos de Moguer,79602,Arganzuela,796,Madrid,0.65,39372.31
10,796025,Delicias,79602,Arganzuela,796,Madrid,1.07,25485.05
8,796023,La Chopera,79602,Arganzuela,796,Madrid,0.56,35276.79
7,796022,Las Acacias,79602,Arganzuela,796,Madrid,1.10,33318.18
6,796021,Imperial,79602,Arganzuela,796,Madrid,0.98,23105.10
127,796215,Corralejos,79621,Barajas,796,Madrid,4.67,1577.09
123,796211,Alameda de Osuna,79621,Barajas,796,Madrid,1.98,9795.45
124,796212,Aeropuerto,79621,Barajas,796,Madrid,19.87,89.88


In [97]:
df.columns

Index(['barrio_codigo', 'barrio_nombre', 'distrito_codigo', 'distrito_nombre',
       'municipio_codigo', 'municipio_nombre', 'superficie_km2',
       'densidad_por_km2'],
      dtype='object')

In [98]:
df.shape

(128, 8)

In [106]:
dfm=df.groupby(['distrito_codigo','distrito_nombre']).agg(','.join)
dfm.reset_index(inplace=True)
dfm.drop(['municipio_nombre'], axis=1, inplace=True)
#dfm.rename(columns={'distrito_codigo': 'DistrictCode', 'distrito_nombre':'DistrictName','barrio_nombre':'Neighborhood'}, inplace=True)
dfm

,distrito_codigo,distrito_nombre,barrio_nombre
0,79601,Centro,"Palacio, Embajadores, Co..."
1,79602,Arganzuela,"Imperial, Las Acacias, L..."
2,79603,Retiro,"Pacífico, Adelfas, Estre..."
3,79604,Salamanca,"Recoletos, Goya, Fuente ..."
4,79605,Chamartín,"El Viso, Prosperidad, Ci..."
5,79606,Tetuan,"Bellas Vistas, Cuatro Caminos, ..."
6,79607,Chamberí,"Gaztambide, Arapiles, Tr..."
7,79608,Fuencarral-El Pardo,"El Pardo, Fuentelarreina, ..."
8,79609,Moncloa-Aravaca,"Casa de Campo, Argüelles, ..."
9,79610,Latina,"Los Cármenes, Puerta del Angel,..."


In [100]:
dfm.shape

(21, 3)

In [109]:
df2=df.groupby(['distrito_codigo','distrito_nombre']).sum()
df2['population']=df2['superficie_km2']*df2['densidad_por_km2']
df2.reset_index(inplace=True)
df2.drop(['distrito_nombre','barrio_codigo','municipio_codigo','superficie_km2','densidad_por_km2'],axis=1,inplace=True)
df

,distrito_codigo,population
0,79601,7.697384e+05
1,79602,1.120038e+06
2,79603,9.043745e+05
3,79604,8.947141e+05
4,79605,9.944694e+05
5,79606,9.706861e+05
6,79607,9.133525e+05
7,79608,1.866089e+07
8,79609,2.805213e+06
9,79610,2.637501e+06


In [ ]:
# Formato poblacion
#df2.style.format("{:.2f}")

In [113]:
dfR = pd.merge(dfm, df2, on='distrito_codigo')
dfR

,distrito_codigo,distrito_nombre,barrio_nombre,population
0,79601,Centro,"Palacio, Embajadores, Co...",7.697384e+05
1,79602,Arganzuela,"Imperial, Las Acacias, L...",1.120038e+06
2,79603,Retiro,"Pacífico, Adelfas, Estre...",9.043745e+05
3,79604,Salamanca,"Recoletos, Goya, Fuente ...",8.947141e+05
4,79605,Chamartín,"El Viso, Prosperidad, Ci...",9.944694e+05
5,79606,Tetuan,"Bellas Vistas, Cuatro Caminos, ...",9.706861e+05
6,79607,Chamberí,"Gaztambide, Arapiles, Tr...",9.133525e+05
7,79608,Fuencarral-El Pardo,"El Pardo, Fuentelarreina, ...",1.866089e+07
8,79609,Moncloa-Aravaca,"Casa de Campo, Argüelles, ...",2.805213e+06
9,79610,Latina,"Los Cármenes, Puerta del Angel,...",2.637501e+06


In [115]:
dfR.shape

(21, 4)